<a href="https://colab.research.google.com/github/vaguiar/CodingSamples/blob/master/src/explore/CDC_provisonal_flow_to_counts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore CDC data and convert the 12month flow data to counts


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# # Uncomment this to read data from Drive
prov_link = '/content/drive/My Drive/Capstone Project 2019/data/cdc-wonder/cdc-provisional-2014-2019.csv'

cdc_state_link = '/content/drive/My Drive/Capstone Project 2019/data/cdc-wonder/cdc_opioids_state.csv'


In [0]:
import pandas as pd
import numpy as np

prov_data = pd.DataFrame(pd.read_csv(prov_link))
cdc_data = pd.DataFrame(pd.read_csv(cdc_state_link))


In [84]:
# Drop rows containing NA in months
cdc_data = cdc_data.dropna(axis=0, how='any', subset=['Month'])
cdc_data

,Notes,State,State Code,Month,Month Code,Deaths,Population,Crude Rate,Unnamed: 8,Unnamed: 9
0,NaN,Alabama,1.0,"Jan., 2014",2014/01,71.0,Not Applicable,Not Applicable,NaN,NaN
1,NaN,Alabama,1.0,"Feb., 2014",2014/02,54.0,Not Applicable,Not Applicable,NaN,NaN
2,NaN,Alabama,1.0,"Mar., 2014",2014/03,79.0,Not Applicable,Not Applicable,NaN,NaN
3,NaN,Alabama,1.0,"Apr., 2014",2014/04,76.0,Not Applicable,Not Applicable,NaN,NaN
4,NaN,Alabama,1.0,"May, 2014",2014/05,59.0,Not Applicable,Not Applicable,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2248,NaN,Wyoming,56.0,"Mar., 2016",2016/03,13.0,Not Applicable,Not Applicable,NaN,NaN
2249,NaN,Wyoming,56.0,"May, 2016",2016/05,13.0,Not Applicable,Not Applicable,NaN,NaN
2250,NaN,Wyoming,56.0,"Jun., 2016",2016/06,10.0,Not Applicable,Not Applicable,NaN,NaN
2251,NaN,Wyoming,56.0,"Oct., 2017",2017/10,13.0,Not Applicable,Not Applicable,NaN,NaN


In [0]:
cdc_last_month = cdc_data.loc[(cdc_data['Month Code'].str.contains('2017', regex=False))]

cdc_last = pd.DataFrame(cdc_last_month, columns=['State', 'Month Code', 'Deaths'])

In [118]:
cdc_last

,State,Month Code,Deaths
36,Alabama,2017/01,91.0
37,Alabama,2017/02,73.0
38,Alabama,2017/03,92.0
39,Alabama,2017/04,75.0
40,Alabama,2017/05,83.0
...,...,...,...
2233,Wisconsin,2017/10,109.0
2234,Wisconsin,2017/11,81.0
2235,Wisconsin,2017/12,97.0
2251,Wyoming,2017/10,13.0


In [119]:
all_states = pd.Series(cdc_data.drop_duplicates('State')['State']).dropna(inplace=False)

print("Has 51 states because it include DC as a state")
print(all_states)

Has 51 states because it include DC as a state
0                    Alabama
48                    Alaska
79                   Arizona
127                 Arkansas
175               California
223                 Colorado
271              Connecticut
319                 Delaware
365     District of Columbia
402                  Florida
450                  Georgia
498                   Hawaii
544                    Idaho
591                 Illinois
639                  Indiana
687                     Iowa
735                   Kansas
783                 Kentucky
831                Louisiana
879                    Maine
927                 Maryland
975            Massachusetts
1023                Michigan
1071               Minnesota
1119             Mississippi
1167                Missouri
1215                 Montana
1248                Nebraska
1290                  Nevada
1338           New Hampshire
1386              New Jersey
1434              New Mexico
1482                New Y

In [68]:
# left_states = all_states[~(all_states.isin(cdc_last['State']))]

# left_states

1215         Montana
1578    North Dakota
1870    South Dakota
2236         Wyoming
Name: State, dtype: object

In [78]:
# left_df = pd.DataFrame(columns=['State', 'Month',	'Month Code',	'Deaths'])

# for st in left_states:
#   ser = {'State' : st, 
#          'Month' : 'Dec., 2017',
#          'Month Code' :'2017/12', 
#          'Deaths' : 0}
#   print("Appending row:", ser)
#   cdc_last = cdc_last.append(ser, ignore_index=True) 

# cdc_last = cdc_last.reindex()

Appending row: {'State': 'Montana', 'Month': 'Dec., 2017', 'Month Code': '2017/12', 'Deaths': 0}
Appending row: {'State': 'North Dakota', 'Month': 'Dec., 2017', 'Month Code': '2017/12', 'Deaths': 0}
Appending row: {'State': 'South Dakota', 'Month': 'Dec., 2017', 'Month Code': '2017/12', 'Deaths': 0}
Appending row: {'State': 'Wyoming', 'Month': 'Dec., 2017', 'Month Code': '2017/12', 'Deaths': 0}


In [0]:
# cdc_last

## Start converting Flow rates to counts in provisional data

In [0]:
# Keep only rows with Indicator = "Number of Drug Overdose Deaths"

new_prov = prov_data.loc[(prov_data['Indicator'] == "Number of Drug Overdose Deaths")]

In [123]:
months = ["January", "February", "March", "April", "May", "June", 
          "July", "August", "September", "October", "November", "December"]


new_prov['Month'] = pd.Categorical(new_prov['Month'], categories=months, ordered=True)

prov_sort = new_prov.sort_values(by=['Year', 'State', 'Month'], inplace=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [133]:
prov_sort = prov_sort.loc[(prov_sort['Year'] >= 2018)]

prov_sort.head(3)

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
483,AK,2018,January,12 month-ending,Number of Drug Overdose Deaths,138,100,0.0,Alaska,Underreported due to incomplete data.,*,138
477,AK,2018,February,12 month-ending,Number of Drug Overdose Deaths,139,100,0.0,Alaska,Underreported due to incomplete data.,*,139
520,AK,2018,March,12 month-ending,Number of Drug Overdose Deaths,136,100,0.0,Alaska,Underreported due to incomplete data.,*,137


In [141]:
cdc_2017_deaths = cdc_last.groupby(by=['State'], axis=0)['Deaths'].sum()

cdc_2017_deaths.head(3)

State
Alabama     960.0
Alaska      125.0
Arizona    1628.0
Name: Deaths, dtype: float64

In [145]:
prov_sort.head(3)

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
483,AK,2018,January,12 month-ending,Number of Drug Overdose Deaths,138,100,0.0,Alaska,Underreported due to incomplete data.,*,138
477,AK,2018,February,12 month-ending,Number of Drug Overdose Deaths,139,100,0.0,Alaska,Underreported due to incomplete data.,*,139
520,AK,2018,March,12 month-ending,Number of Drug Overdose Deaths,136,100,0.0,Alaska,Underreported due to incomplete data.,*,137


In [151]:
prov_sort = prov_sort.loc[(prov_sort['Year'] >= 2018)]

for row in prov_sort.iterrows():
  print(row)
  if row['Month'] == 'January' and row['Year'] == '2018':

    state = row['State']
    print('State: ', state)
    print('deaths in Jan 2017: ', cdc_last['Deaths'][state])
    # feb_2_dec = cdc_2017_deaths[state] - cdc_last['Deaths']['']

(483, State                                                               AK
Year                                                              2018
Month                                                          January
Period                                                 12 month-ending
Indicator                               Number of Drug Overdose Deaths
Data Value                                                         138
Percent Complete                                                   100
Percent Pending Investigation                                        0
State Name                                                      Alaska
Footnote                         Underreported due to incomplete data.
Footnote Symbol                                                      *
Predicted Value                                                    138
Name: 483, dtype: object)


TypeError: ignored